In [2]:
csv_path = '《乘风破浪的姐姐》成团之夜-1.csv'
stop_words_path = 'cn_stopwords.txt'

In [3]:
###### 模型训练

In [4]:
%%time
import pandas as pd

test = pd.read_csv(csv_path,encoding = "gb18030",warn_bad_lines=False,error_bad_lines=False)                                     ################ 数据集位置
test_data = pd.DataFrame(test)
# 打乱数据集
re_test_data = test.sample(frac=1).reset_index(drop=True)

####去除特殊符号并分词####
import jieba
import re
# 使用jieba进行分词
def chinese_word_cut(mytext):
    # 去除[@用户]避免影响后期预测精度  
    mytext = re.sub(r'@\w+','',mytext)
    # 去除数字字母的字符串
    mytext = re.sub(r'[a-zA-Z0-9]','',mytext)
    return " ".join(jieba.cut(mytext))
# apply的方法是将数据着行处理
re_test_data['cut_review'] = re_test_data.全文内容.apply(chinese_word_cut)          ################# 数据集内容列名字

####停用词处理####
import re
# 获取停用词列表
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding='utf-8') as f:                         
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
cachedStopWords = get_custom_stopwords(stop_words_path)                              #################停用词文件位置
# 去除停用词方法
def remove_stropwords(mytext):
    return " ".join([word for word in mytext.split() if word not in cachedStopWords])
re_test_data['remove_strop_word'] = re_test_data.cut_review.apply(remove_stropwords)

####保存数据####
# 截取处理后的评论数据和标签
re_data = re_test_data.loc[:,['remove_strop_word','微博情绪']]                             ################### 数据集标签列名字
# 将数据保存为新的csv
re_data.to_csv ("re_sentiment_data.csv" , encoding = "utf_8_sig")

####数据分割####
X = re_test_data['remove_strop_word']
y = re_test_data.微博情绪
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)






# 加载模型及保存模型
import joblib
# 朴素贝叶斯算法
from sklearn.naive_bayes import MultinomialNB
# TFIDF模型
from sklearn.feature_extraction.text import TfidfVectorizer
# 管道模型可将两个算法进行连接
from sklearn.pipeline import Pipeline
# SVM
from sklearn.svm import SVC


####使用SVM进行训练####
TFIDF_SVM_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('SVM', SVC(C=0.95,kernel="linear",probability=True))
])
TFIDF_SVM_Sentiment_Model.fit(X_train[:30000],y_train[:30000])
svm_test_score = TFIDF_SVM_Sentiment_Model.score(X_test,y_test)
joblib.dump(TFIDF_SVM_Sentiment_Model, 'tfidf_svm1_sentiment.model')
print(svm_test_score)


####使用NBM进行训练
TFIDF_NB_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('NB', MultinomialNB())
])
# 取数据进行训练
nbm = TFIDF_NB_Sentiment_Model.fit(X_train[:80000],y_train[:80000])
nb_train_score = TFIDF_NB_Sentiment_Model.score(X_test,y_test)
joblib.dump(TFIDF_NB_Sentiment_Model, 'tfidf_nb_sentiment.model')
print(nb_train_score)

<timed exec>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


<timed exec>:3: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3f/hj3gtg8n3gz5xm84jn7ywct40000gn/T/jieba.cache
Loading model cost 0.683 seconds.
Prefix dict has been built successfully.
<timed exec>:70: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.


0.7181733849342667


<timed exec>:82: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.


0.6803786001072604
CPU times: user 37min 34s, sys: 15.3 s, total: 37min 49s
Wall time: 38min 1s


In [ ]:
###### 使用模型进行情感打分

In [7]:
import re
import joblib
# 获取停用词列表
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding='utf-8') as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
 
 
# 去除停用词方法
def remove_stropwords(mytext,cachedStopWords):
    return " ".join([word for word in mytext.split() if word not in cachedStopWords])
 
# 处理否定词不的句子
def  Jieba_Intensify(text):
    word = re.search(r"不[\u4e00-\u9fa5 ]",text)
    if word!=None:
        text = re.sub(r"(不 )|(不[\u4e00-\u9fa5]{1} )",word[0].strip(),text)
    return text
 
# 判断句子消极还是积极
def IsPoOrNeg(text):
    # 加载训练好的模型     
#     model = joblib.load('tfidf_nb_sentiment.model')
    model = joblib.load('tfidf_svm1_sentiment.model')
    # 获取停用词列表   
    cachedStopWords = get_custom_stopwords(stop_words_path)
    # 去除停用词    
    text = remove_stropwords(text,cachedStopWords)
    # jieba分词         
    seg_list = jieba.cut(text, cut_all=False)
    text = " ".join(seg_list)
    # 否定不处理
    text = Jieba_Intensify(text)
    y_pre =model.predict([text])
    proba = model.predict_proba([text])[0]
    if y_pre[0]==1:
        print(text,"\n\n积极情绪，概率："+str(proba[1]))
    else:
        print(text,"\n\n消极情绪,概率："+str(proba[0]))
 
IsPoOrNeg("真漂亮，云南确实是个好地方啊")

真 漂亮 ， 云南 确实 是 个 好 地方 啊 

积极情绪，概率：0.9711806327772461
